# Paleo-Detector Analysis Pipeline

This notebook provides a complete, end-to-end workflow for the phenomenological analysis of paleo-detectors. It is designed to calculate the expected nuclear recoil track spectra from both external cosmic ray muon signals and various internal/atmospheric backgrounds.

The pipeline is built around the *mineral_utils.py* module, which contains the *Paleodetector* class. This class acts as the physics engine, handling all low-level data processing and calculations. The notebook serves as a high-level interface to configure and orchestrate the analysis.

### Workflow Overview:

**1. Configuration:** All analysis parameters are defined in the configuration cells. This includes selecting the mineral from a library, defining the geological history of the sample (age, exposure time, deposition rate), and setting up the astrophysical scenarios for the cosmic ray flux.

**2. Initialization:** A Paleodetector object is created. This object automatically loads and caches all necessary input data, such as SRIM tables for ion ranges and pre-computed Geant4 results for muon interactions.

**3. Background Calculation:** The notebook calculates the track spectra for the primary background sources:

- Spontaneous Fission: Tracks from the spontaneous fission of Uranium-238 impurities within the mineral.

- Radiogenic Neutrons: Tracks from neutrons produced by (α,n) reactions in the surrounding rock.

- Neutrinos: Tracks from coherent elastic neutrino-nucleus scattering.

**7. Signal Calculation:** For each defined astrophysical scenario, the notebook calculates the track spectrum from cosmic ray muons. This core step includes a sophisticated model for a time-variant cosmic ray flux and the continuous deposition of overburden, which attenuates the signal over the exposure window. The time integration is performed in parallel for efficiency.

**8. Analysis & Plotting:** The final signal and background components are combined to produce the summary plots, showing the total expected number of tracks as a function of track length.


In [18]:
import yaml
import numpy as np
from matplotlib import pyplot as plt

# --- Custom Utility Module ---
from mineral_utils import Paleodetector

print("Libraries imported successfully.")

Libraries imported successfully.


In [19]:
mineral_name =  "Halite"  # Must match a key in MINERAL_LIBRARY

sample_mass_kg = 0.0005       # Sample mass in kg (e.g., 0.01 for 10g)
total_age_myr = 5.6
total_exposure_kyr = 5600.         # Total age of the sample in millions of years
exposure_window_kyr = 300.  # Duration of CR signal exposure in kyr

# --- Continuous Deposition Model Parameters ---
deposition_rate_m_kyr = np.array([2.5, 20.])
continuous_overburden_density_g_cm3 = 2.16
event_overburden_density_g_cm3 = 1.
event_overburden_depth = 1500.
initial_depth_m =  0.


x_bins = np.linspace(0, 100000, 100)        #Track length bins in nm
x_mids = x_bins[:-1] + np.diff(x_bins) / 2

In [20]:
CONFIG = yaml.safe_load(open('Data/basic_config.yaml', 'r'))
MINERAL_LIBRARY = yaml.safe_load(open('Data/mineral_library.yaml', 'r'))

In [21]:
scenario_config_simple = {
    'name': 'simple',
    'event_fluxes': {
        0.: ('H3a', 'H3a'),
        300.: ('H3a', 'H3a'),
    }
}

scenario_config_SN20 = {
    'name': 'SN20',
    'event_fluxes': {
        0.: ('SN20pc_2kyr', 'SN20pc_2kyr'),            
        10.: ('SN20pc_12kyr', 'SN20pc_12kyr'),
        20.: ('SN20pc_22kyr', 'SN20pc_22kyr'),
        30.: ('SN20pc_32kyr', 'SN20pc_32kyr'),
        40.: ('SN20pc_42kyr', 'SN20pc_42kyr'),
        50.: ('SN20pc_52kyr', 'SN20pc_52kyr'),
        60.: ('SN20pc_62kyr', 'SN20pc_62kyr'),
        70.: ('H3a', 'H3a'),
        300.: ('H3a', 'H3a'),         
    }
}

scenario_config_SN100 = {
    'name': 'SN100',
    'event_fluxes': {
        0.: ('SN100pc_2kyr', 'SN100pc_2kyr'),            
        10.: ('SN100pc_12kyr', 'SN100pc_12kyr'),
        20.: ('SN100pc_22kyr', 'SN100pc_22kyr'),
        30.: ('SN100pc_32kyr', 'SN100pc_32kyr'),
        40.: ('SN100pc_42kyr', 'SN100pc_42kyr'),
        50.: ('H3a', 'H3a'),
        300.: ('H3a', 'H3a'),   
    }
}

In [22]:
# --- Setup & Verification ---
mineral_config = MINERAL_LIBRARY.get(mineral_name)
if not mineral_config:
    raise ValueError(f"Mineral '{mineral_name}' not found in MINERAL_LIBRARY.")

mineral = Paleodetector(mineral_config)

Initialized Paleodetector: Halite


## Calculation of Track Spectra

With the Paleodetector object initialized and all necessary data loaded, we can now proceed with the core physics calculations. The following cells will compute the differential track rate (dR/dx) for each relevant physical process and then integrate these rates over the sample's lifetime to find the total expected number of tracks in each track length bin.

### Calculation Steps:

**1. Backgrounds:** We first calculate the spectra for the steady-state backgrounds that accumulate over the entire geological age of the sample. These include spontaneous fission, radiogenic neutrons, and atmospheric neutrinos.

**2. Muon Signal:** Next, we compute the primary signal from cosmic ray muons. The integrate_muon_signal_spectrum_parallel function is used for this. It performs a numerical integration over the specified exposure_window_kyr, accounting for two key time-dependent effects at each step:

- The evolution of the cosmic ray flux, which is interpolated from the files defined in the scenario_config.

- The increase in shielding due to the continuous deposition_rate_m_kyr.

**3. Final Plot:** Finally, all calculated track components (signals and backgrounds) are summed and plotted to visualize the expected final track spectrum in the detector.

In [ ]:
total_fission_tracks = mineral.integrate_fission_spectrum(x_bins, total_age_myr, sample_mass_kg)

Calculating spontaneous fission background...


In [ ]:
total_neutron_tracks = mineral.integrate_neutron_spectrum(x_bins, total_age_myr, sample_mass_kg)

In [ ]:
total_muon_tracks_simple = [mineral.integrate_muon_signal_spectrum_parallel(
    x_bins, 
    scenario_config=scenario_config_simple, 
    energy_bins_gev=CONFIG["geant4_energy_bins_gev"], 
    exposure_window_kyr=exposure_window_kyr, 
    sample_mass_kg = sample_mass_kg,
    deposition_rate_m_kyr = deposition,
    overburden_density_g_cm3 = continuous_overburden_density_g_cm3,
    nsteps=20,
) for deposition in deposition_rate_m_kyr]

In [ ]:
total_muon_tracks_SN20 = [mineral.integrate_muon_signal_spectrum_parallel(
    x_bins, 
    scenario_config=scenario_config_SN20, 
    energy_bins_gev=CONFIG["geant4_energy_bins_gev"], 
    exposure_window_kyr=exposure_window_kyr, 
    sample_mass_kg = sample_mass_kg,
    deposition_rate_m_kyr=deposition,
    overburden_density_g_cm3=continuous_overburden_density_g_cm3,
    nsteps=150,
) for deposition in deposition_rate_m_kyr]

In [ ]:
total_muon_tracks_SN100 = [mineral.integrate_muon_signal_spectrum_parallel(
    x_bins, 
    scenario_config=scenario_config_SN100, 
    energy_bins_gev=CONFIG["geant4_energy_bins_gev"], 
    exposure_window_kyr=exposure_window_kyr, 
    sample_mass_kg = sample_mass_kg,
    deposition_rate_m_kyr = deposition,
    overburden_density_g_cm3 = continuous_overburden_density_g_cm3,
    nsteps=150,
) for deposition in deposition_rate_m_kyr]

In [ ]:
post_tracks = [mineral.integrate_muon_signal_spectrum_parallel(
    x_bins,
    scenario_config=scenario_config_simple,
    energy_bins_gev=CONFIG["geant4_energy_bins_gev"],
    exposure_window_kyr = total_exposure_kyr - exposure_window_kyr,
    sample_mass_kg = sample_mass_kg,
    initial_depth = event_overburden_depth + deposition * exposure_window_kyr * continuous_overburden_density_g_cm3,
    nsteps=5,
) for deposition in deposition_rate_m_kyr]

In [ ]:
fig, ax = plt.subplots(figsize=[7, 5])

i = 1

ax.step(x_mids/1000, np.array(total_fission_tracks)/(1e3*sample_mass_kg), color='C3', linestyle='dashed', linewidth=1., label="Fission tracks", where='mid')
ax.step(x_mids/1000, np.array(total_neutron_tracks)/(1e3*sample_mass_kg), color='C5', linestyle='dashed', linewidth=1., label="Neutron-induced tracks", where='mid')
ax.step(x_mids/1000, np.array(total_muon_tracks_simple[i])/(1e3*sample_mass_kg), color='C0', label="Muon-induced tracks: Simple Scenario", linestyle='-', linewidth=1., where='mid')
ax.step(x_mids/1000, np.array(total_muon_tracks_SN20[i])/(1e3*sample_mass_kg), color='C1', label="Muon-induced tracks: SN20 Scenario", linestyle='-', linewidth=1., where='mid')
ax.step(x_mids/1000, np.array(total_muon_tracks_SN100[i])/(1e3*sample_mass_kg), color='C2', label="Muon-induced tracks: SN100 Scenario", linestyle='-', linewidth=1., where='mid')
ax.step(x_mids/1000, np.array(post_tracks[i])/(1e3*sample_mass_kg), color='C4', label="Muon-induced tracks: with overburden", linestyle='-', linewidth=1., where='mid')

ax.set_title(f"Track density distribution in {mineral_name} in the slow deposition scenario")
ax.set_xlabel(r"Track Length ($\mu$m)")
ax.set_ylabel(r"Density of Tracks (g$^{-1}$)")
ax.set_yscale("log")
ax.set_ylim(1e-2, 1e8)
ax.set_xlim(0, 50)
ax.legend(fontsize='small')
plt.savefig("Plots/MSC_track_number_per_gram.png", dpi=500)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=[7, 5])

ax.step(x_mids/1000, np.array(total_fission_tracks), color='C3', linestyle='dashed', linewidth=1., label="Fission tracks", where='mid')
ax.step(x_mids/1000, np.array(total_neutron_tracks), color='C5', linestyle='dashed', linewidth=1., label="Neutron-induced tracks", where='mid')

for i in range(len(deposition_rate_m_kyr)):
    ax.plot(x_mids/1000, np.array(total_muon_tracks_simple[i]), color='C0', label=f"Muon-induced tracks: Simple Scenario, deposition rate {deposition_rate_m_kyr[i]}", linestyle='-', linewidth=1.)
    ax.plot(x_mids/1000, np.array(total_muon_tracks_SN20[i]), color='C1', label=f"Muon-induced tracks: SN20 Scenario, deposition rate {deposition_rate_m_kyr[i]}", linestyle='-', linewidth=1.)
    ax.plot(x_mids/1000, np.array(total_muon_tracks_SN100[i]), color='C2', label=f"Muon-induced tracks: SN100 Scenario, deposition rate {deposition_rate_m_kyr[i]}", linestyle='-', linewidth=1.)
    ax.plot(x_mids/1000, np.array(post_tracks[i]), color='C4', label=f"Muon-induced tracks: with overburden, deposition rate {deposition_rate_m_kyr[i]}", linestyle='-', linewidth=1.)

ax.fill_between(x_mids/1000, total_muon_tracks_simple[0], total_muon_tracks_simple[-1], color='C0', alpha=0.2, label='Simple Scenario Range')
ax.fill_between(x_mids/1000, total_muon_tracks_SN20[0], total_muon_tracks_SN20[-1], color='C0', alpha=0.2, label='SN20 Scenario Range')
ax.fill_between(x_mids/1000, total_muon_tracks_SN100[0], total_muon_tracks_SN100[-1], color='C0', alpha=0.2, label='SN100 Scenario Range')
ax.fill_between(x_mids/1000, post_tracks[0], post_tracks[-1], color='C4', alpha=0.2, label='Post-Flood Scenario Range')

ax.set_title(f"Track density distribution in {mineral_name} in the two deposition scenarios")
ax.set_xlabel(r"Track Length ($\mu$m)")
ax.set_ylabel(r"Density of Tracks (g$^{-1}$)")
ax.set_yscale("log")
ax.set_ylim(1e-5, 1e8)
ax.set_xlim(0, 50)
ax.legend(fontsize='small')
plt.savefig("Plots/MSC_track_number_depositions.png", dpi=500)
plt.show()

In [ ]:
filepath = './Data/processed_recoils/Halite_muon_recoil_simple_0.0kyr_0mwe.npz'
recoil_data = np.load(filepath)

drdx_muon = mineral._convert_recoil_to_track_spectrum(x_bins=x_bins, recoil_data=recoil_data, energy_bins_gev=CONFIG["geant4_energy_bins_gev"])
drdx_fission = mineral.calculate_fission_spectrum(x_bins)
drdx_neutron = mineral.calculate_neutron_spectrum(x_bins)

drdx_org = np.zeros_like(drdx_muon['total'])
drdx_result = np.zeros_like(drdx_muon['total'])
for name in recoil_data.files[1:]:
    if name != 'total' and name != 'Er_bins':
        if name in ['Na', 'Cl']:
            drdx_org += drdx_muon[name]
        else: drdx_result += drdx_muon[name]

In [ ]:
filepath_SN20 = './Data/processed_recoils/Halite_muon_recoil_SN20_0.0kyr_0mwe.npz'
recoil_data_SN20 = np.load(filepath_SN20)

drdx_muon_SN20 = mineral._convert_recoil_to_track_spectrum(x_bins=x_bins, recoil_data=recoil_data_SN20, energy_bins_gev=CONFIG["geant4_energy_bins_gev"])

filepath_SN100 = './Data/processed_recoils/Halite_muon_recoil_SN100_0.0kyr_0mwe.npz'
recoil_data_SN100 = np.load(filepath_SN100)

drdx_muon_SN100 = mineral._convert_recoil_to_track_spectrum(x_bins=x_bins, recoil_data=recoil_data_SN100, energy_bins_gev=CONFIG["geant4_energy_bins_gev"])

filepath_over = './Data/processed_recoils/Halite_muon_recoil_simple_0.0kyr_4740mwe.npz'
recoil_data_over = np.load(filepath_over)

drdx_muon_over = mineral._convert_recoil_to_track_spectrum(x_bins=x_bins, recoil_data=recoil_data_over, energy_bins_gev=CONFIG["geant4_energy_bins_gev"])


In [ ]:
fig, ax = plt.subplots(figsize=[7, 5])

ax.plot(x_mids, drdx_muon['total'], label='Total muon-induced')
ax.plot(x_mids, drdx_muon_SN20['total'], linestyle='-', linewidth=1., label='Total muon-induced at peak SN20')
ax.plot(x_mids, drdx_muon_SN100['total'], linestyle='-', linewidth=1., label='Total muon-induced at peak SN100')

ax.plot(x_mids, drdx_muon_over['total'], linestyle='--', linewidth=1., label='Total muon-induced with overburden')

ax.plot(x_mids, drdx_org, linestyle='-.', linewidth=0.6, color='C5', label='Target nuclides only')
ax.plot(x_mids, drdx_result, linestyle='-.', linewidth=0.6, color='C6', label='Nuclides from spallation')
ax.plot(x_mids, drdx_neutron, linestyle='-', color = 'C4',label='Neutron-induced')
ax.plot(x_mids, drdx_fission, linestyle='-', color = 'C2',label='Fission-induced')

ax.set_yscale("log")
ax.set_xscale("log")
ax.set_xlabel(r"Track Length (nm)")
ax.set_ylabel(r"dR/dx (nm$^{-1}$ kg$^{-1}$ Myr$^{-1}$)")
#ax.set_title(f"Track rate spectrum in {mineral_name}")
ax.set_ylim(1e-2, 3e10)
ax.legend(fontsize='small', loc='upper right')
plt.savefig("Plots/MSC_drdx.png", dpi=500)
plt.show()